In [2]:
import pandas as pd
import re
from glob import glob

# 1) preprocessing_all.csv 불러오기
df_pre = pd.read_csv('preprocessing_all.csv')

# 2) 조회일에서 리스트로 날짜 뽑기 함수
def parse_dates(cell):
    if isinstance(cell, str) and cell.strip().startswith('['):
        # 리스트 형태 문자열에서 YYYY-MM-DD 패턴 전부 추출
        return re.findall(r"(\d{4}-\d{2}-\d{2})", cell)
    else:
        # 단일 날짜 문자열 혹은 Timestamp
        m = re.search(r"(\d{4}-\d{2}-\d{2})", str(cell))
        return [m.group(1)] if m else []

# 3) date_list & base_ym_list 컬럼 생성
df_pre['date_list'] = df_pre['조회일'].apply(parse_dates)
df_pre['base_ym_list'] = df_pre['date_list'].apply(
    lambda dl: [int(pd.to_datetime(d).strftime('%Y%m')) for d in dl]
)

# 4) 문화지수 데이터 로드
files = glob('문화지수데이터/KC_CLTUR_MOVIE_ACTIVE_IDEX_LIST_*.csv')
df_idx = pd.concat(
    (pd.read_csv(f, dtype={'BASE_YM': int}) for f in files),
    ignore_index=True
)

# 5) '서울시' 행만 필터
df_seoul = df_idx[df_idx['CTPRVN_NM'] == '서울시'].copy()

# 6) 서울시 지표 컬럼 목록 추출
exclude_cols = ['BASE_YM','BASE_YEAR','BASE_MT','CTPRVN_CD','CTPRVN_NM']
metrics = [c for c in df_seoul.columns if c not in exclude_cols]

# 7) BASE_YM을 인덱스로 설정
df_seoul_wide = df_seoul.set_index('BASE_YM')[metrics]

# 8) df_pre 각 행에 서울시 지표 리스트로 붙이기
for m in metrics:
    df_pre[f'{m}_seoul_list'] = df_pre['base_ym_list'].apply(
        lambda yms: [
            df_seoul_wide.loc[ym, m] if ym in df_seoul_wide.index else None
            for ym in yms
        ]
    )

# 9) (선택) 중간 컬럼 제거
df_pre = df_pre.drop(columns=['date_list', 'base_ym_list'])

# 10) 결과 확인
print("최종 데이터프레임 크기:", df_pre.shape)
print("컬럼 목록:", df_pre.columns.tolist())

최종 데이터프레임 크기: (6650, 34)
컬럼 목록: ['영화명', '매출액증감', '관객수증감(전일대비)', '매출액', '관객수', '스크린수', '상영횟수', '조회일', '개봉일', '대표국적', '국적', '배급사', '등급', '장르', '감독', '매출액(점유율)', '누적매출액', '누적관객수', '중복횟수', '순위', '매출액증감율(전일대비)', '관객수증감율(전일대비)', '제작사', '배우', 'SCRNG_MOVIE_CO_seoul_list', 'MOVIE_ADNC_CO_seoul_list', 'EXPNDTR_PRICE_seoul_list', 'POPLTN_PER_MOVIE_ADNC_CO_seoul_list', 'MOVIE_ADNC_CO_SCORE_seoul_list', 'EXPNDTR_PRICE_SCORE_seoul_list', 'GNRLZ_SCORE_seoul_list', 'STDR_SCRNG_MOVIE_CO_seoul_list', 'STDR_MOVIE_ADNC_CO_seoul_list', 'STDR_EXPNDTR_PRICE_seoul_list']


In [3]:
# 10) 결과를 CSV로 저장
df_pre.to_csv('preprocessing_all_with_seoul.csv', index=False, encoding='utf-8-sig')